In [40]:
import os
import pandas as pd
import numpy as np
import re
import datetime
import pickle
import json
import time
from nltk.tokenize import sent_tokenize

os.chdir('..')
print(os.getcwd())
from utils import get_fulltext,get_fname,fulltext_exists
os.chdir('./data_processing/')

/Users/yiweiluo/scientific-debates


In [2]:
df = pd.read_pickle('../data_scraping/dedup_combined_df.pkl')
df.head(10)

,url,title,date,domain,stance,topic,is_AP,year,pretty_domain
0,www.buzzfeednews.com/article/tasneemnashrulla/...,"""eat the babies"" viral video at aoc town hall ...",2019-10-04 00:00:00,buzzfeed,pro,cc,None,2019.0,Buzzfeed
1,www.buzzfeednews.com/article/passantino/extrem...,"""extremely likely"" global warming is man-made,...",2013-09-27 00:00:00,buzzfeed,pro,cc,None,2013.0,Buzzfeed
2,shotofprevention.com/2010/11/03/history-makes-...,"""history"" makes headlines with launch of new w...",2020-03-13 14:32:02,https://shotofprevention/,pro,vax,False,2020.0,Shot of Prevention
3,www.buzzfeednews.com/article/andrewkaczynski/i...,"""it's global warming, stupid"" - buzzfeed news",2012-11-01 00:00:00,buzzfeed,pro,cc,None,2012.0,Buzzfeed
4,www.buzzfeednews.com/article/tasneemnashrulla/...,"""japan dropped an atomic bomb on america durin...",2014-02-24 00:00:00,buzzfeed,pro,cc,None,2014.0,Buzzfeed
5,www.buzzfeednews.com/article/llevin/opinion-im...,"""look at my record, child"": joe biden showed m...",2019-10-31 00:00:00,buzzfeed,pro,cc,None,2019.0,Buzzfeed
6,www.breitbart.com/politics/2019/06/14/orourke-...,"""president o'rourke will end oil and gas lease...",2019-06-14 00:00:00,breitbart,anti,cc,None,2019.0,Breitbart
7,www.buzzfeednews.com/article/andrewkaczynski/s...,smoking doesnt kill and other great old opeds ...,2015-03-31 00:00:00,buzzfeed,pro,cc,None,2015.0,Buzzfeed
8,www.foxnews.com/world/100-carbon-tax-by-2030-c...,"$100 carbon tax by 2030 could save climate, sa...",2017-05-29 00:00:00,fox,anti,cc,None,2017.0,Fox
21585,childrenshealthdefense.org/news/4-billion-and-...,$4 billion and growing: u.s. payouts for vacc...,2018-11-19 00:00:00,chd,anti,vax,None,2018.0,Children's Health Defense


In [3]:
get_fulltext(df.url.values[0])

['The president and the far right slammed Rep. Alexandria Ocasio-Cortez over the bizarre town hall speaker who suggested eating babies to solve the climate crisis. But the pro-Trump LaRouche PAC, which believes climate change is a hoax, said it planted the troll. President Donald Trump, his son Don Jr., and far-right conservatives attacked Democratic Rep. Alexandria Ocasio-Cortez after a speaker at her town hall meeting Thursday night went on a bizarre rant about how eating babies is the only solution to the climate crisis. But it turns out the stunt was staged by what is now a far-right pro-Trump conspiracy group that compares climate change activism to “genocide.” Trump called Ocasio-Cortez a “wack job” on Twitter after his allies widely shared the video of the unidentified woman saying that Ocasio-Cortez’s Green New Deal was not enough to solve the climate crisis. AOC is a Wack Job! https://t.co/LU3hIeek0c  “We’re not going to be here for much long because of the climate crisis,” th

# Contents:
* [Define functions](#Define-functions)
* [Extract clausal complements with SpaCy](#Extract-embedded-clauses-with-SpaCy)

# Define functions

We add a neural coref resolution step to the default SpaCy pipeline: https://github.com/huggingface/neuralcoref

In [9]:
import spacy
nlp = spacy.load('en_core_web_sm')

# First way we can control a parameter
#neuralcoref.add_to_pipe(nlp, greedyness=0.75)
import neuralcoref
neuralcoref.add_to_pipe(nlp)

Helper functions for the main ```get_quotes``` function:

In [4]:
def is_good_verb_dep(dep):
    return dep[:3] == 'aux' or dep[:3] == 'adv' or dep == 'det' or dep == 'rel' or dep == 'prep' or dep[-3:] == 'obj' or dep[-3:] == 'mod' or dep == 'prt' or (dep[-4:] == 'comp' and dep != 'ccomp')

def is_good_subj_dep(dep):
    return dep != 'ccomp'

def is_ROOT(tok):
    return tok.dep_ == 'ROOT' or tok.dep_[-2:] == 'cl' or tok.dep_ == 'ccomp' or \
            (tok.dep_ == 'conj' and tok.head.dep_ == 'ROOT')

REL_PRONOUNS = set(['who', 'whom', 'whose', 'which', 'that'])
def is_rel_pronoun(tok):
    tok = tok.lower().strip()
    return tok in REL_PRONOUNS

In [46]:
def get_quotes(text):
    # Do coref resolution
    doc = nlp(text)
    text = doc._.coref_resolved
    
    quote_objs = []
    
    for sent in sent_tokenize(text):#d.sents:
        #print(sent)
        sent = nlp(sent)
        # Go through entire sentence, looking for verbs embedding a complement clause
        VERBS = np.unique([token.head for token in sent if token.dep_ == 'ccomp'])
        #print(VERBS)
       
        # Extract everything else for each VERB
        for VERB in VERBS:
            #print("\nCcomp dependency found! For quoting verb '{}'".format(VERB))
            # Extract the rest of the quoting verb
            verb_deps = [x for x in VERB.children if is_good_verb_dep(x.dep_)]
            #print("\tFound children:",verb_deps)
            for x in verb_deps:
                new_children = [c for c in x.children if is_good_verb_dep(c.dep_)]
                #print("\tAdding children of {}:".format(x.text),new_children)
                verb_deps.extend(new_children)
                #print("\tUpdated verb deps:",verb_deps)
            
            # If verb's head is not itself (i.e., verb is not the ROOT), 
            # recursively trace back to ROOT, then add all children of ROOT
            ROOT = VERB
            while not is_ROOT(ROOT):
                ROOT = ROOT.head
                #print('\t\tCurrent root:',ROOT)
            if VERB is not ROOT:
                verb_deps.append(ROOT) 
                
            #print("\tAdding children of ROOT...")
            root_deps = [x for x in ROOT.children if is_good_verb_dep(x.dep_)]
            #print("\tFound ROOT deps:",root_deps)
            for x in root_deps:
                new_children = [c for c in x.children if is_good_verb_dep(c.dep_)]
                #print("\tAdding children of {}:".format(x.text),new_children)
                root_deps.extend(new_children)
                #print("\tUpdated ROOT deps:",root_deps)

            #print("\tAdding ROOT deps to verb deps...")
            verb_deps.extend([x for x in root_deps if x != VERB and x not in verb_deps])
            #print("\tUpdated verb deps:",verb_deps)
            
            NEG,IS_NEG,neg_children = None,None,None
            SUBJECT,subj_children = None,None
                
            #print("\nLooking for SUBJECT and NEGATION(s)...")
            for child in ROOT.children:
                if child.dep_[:5] == 'nsubj' or child.dep_ == 'expl':
                    SUBJECT = child
                    #print("\tFound SUBJECT:",SUBJECT)
                    if SUBJECT.head.dep_ == 'relcl' and is_rel_pronoun(SUBJECT.text): # we're dealing with the subject of a rel clause
                        #print("\tFound quote inside a relative clause. Finding antecedent subject...")
                        SUBJECT = SUBJECT.head.head
                        #print("\tTrue subject:",SUBJECT)
                    #print("Subject token '{}' is in a coref cluster:".format(SUBJECT),SUBJECT._.in_coref)
                
                if child.dep_[:3] == 'neg':
                    NEG = child
                    verb_deps.append(NEG)
                    neg_children = [c for c in NEG.children if c != VERB]
                    #print("n\tAdding new NEG children:",neg_children)
                    for x in neg_children:
                        new_children = [c for c in x.children]
                        #print("\tNew NEG grandchildren:",new_children)
                        neg_children.extend(new_children)
                        #print("\tUpdated neg_children:",neg_children)
                    verb_deps.extend(neg_children)
                    #print("\tUpdated verb_deps:",verb_deps)
                    IS_NEG = VERB in NEG.head.children or VERB == NEG.head
            
            if SUBJECT is None and (ROOT.dep_ == 'acl' or ROOT.dep_ == 'advcl'):
                main_verb = ROOT.head
                #print([(c.text,c.dep_) for c in main_verb.children])
                SUBJS = [c for c in main_verb.children if c.dep_[:5] == 'nsubj' or c.dep_ == 'expl']
                SUBJECT = SUBJS[0] if len(SUBJS) > 0 else None
           
            # Get rest of subject tokens
            if SUBJECT is not None:
                #print("\nFound SUBJECT:",SUBJECT)
                #print("\tAdding children of SUBJECT...")
                subj_children = [c for c in SUBJECT.children if is_good_subj_dep(c.dep_)]
                #print("\tFound children:",subj_children)
                for x in subj_children:
                    new_children = [c for c in x.children if is_good_subj_dep(c.dep_)]
                    #print("\tAdding children of child {}:".format(x.text),new_children)
                    subj_children.extend(new_children)
                    #print("\tUpdated subject children:",subj_children)

            sorted_verb_tokens = sorted([(c,c.i) for c in verb_deps+[VERB]],key=lambda x:x[1])
            #print("\n\tSorted verb tokens:",sorted_verb_tokens)
            if SUBJECT is not None:
                sorted_subj_tokens = sorted([(c,c.i) for c in subj_children+[SUBJECT]],key=lambda x:x[1])
                #print("\tSorted subject tokens:",sorted_subj_tokens)
            else:
                sorted_subj_tokens = None
            if NEG is not None:
                sorted_neg_tokens = sorted([(c,c.i) for c in neg_children+[NEG]],key=lambda x:x[1])
            else:
                sorted_neg_tokens = None

            #print("\nFinding quote introduced by '{}'...".format(VERB))
            emb_main_verbs = [c for c in VERB.children if c.dep_ == 'ccomp']
            #print("\tMain verbs of embedded clause:",emb_main_verbs)
            #assert len(emb_main_verbs) <= 2
            for emb_main_verb in emb_main_verbs:
                #print("\tMain *verb*:",emb_main_verb)

                # Recursively get all children of main verb of embedded clause 
                #print("\nRecursively getting children of main verb of embedded clause...\n")
                #print("*"*50)
                children_queue = [x for x in emb_main_verb.children]
                #print("\tAdding children of main verb:",children_queue)
                for x in children_queue:
                    new_children = [c for c in x.children]
                    #print("\tAdding children of {}:".format(x.text),new_children)
                    children_queue.extend(new_children)
                    #print("\tNew children queue:",children_queue)

                # Sort children and matrix verb to be in correct order
                #print("\nSorting tokens in quote...")
                children_and_indices = [(c,c.i) for c in children_queue+[emb_main_verb]]
                sorted_ = sorted(children_and_indices,key=lambda x:x[1])
                quote_objs.append({'quote':[tup[0].text for tup in sorted_],
                        'verb tokens':[tup[0].text for tup in sorted_verb_tokens],
                        'main verb':VERB.text,
                       'subject tokens':[tup[0].text for tup in sorted_subj_tokens] if sorted_subj_tokens is not None else None,
                       'main subject':SUBJECT.text if SUBJECT is not None else None,
                       'neg tokens':[tup[0].text for tup in sorted_neg_tokens] if sorted_neg_tokens is not None else None,
                       'main neg':NEG.text if NEG is not None else None,
                       'is neg':IS_NEG})
    
    return quote_objs

# Extract clausal complements with SpaCy

In [42]:
s = time.time()

In [43]:
time.time()-s

6.651726722717285

## First 100 URLs

In [47]:
for url_ix in range(0,1):
    start_time = time.time()
    curr_url = df.url.values[url_ix]
    quotes = get_quotes(get_fulltext(curr_url)[0])
    fname = get_fname(curr_url)
    with open('./extracted_quotes/{}.jsonlist'.format(fname),'w+') as f:
        for res in quotes:
            json.dump(res, f)
            f.write('\n')
    print('{}\tElapsed time in seconds:'.format(fname),(time.time()-start_time)/60.)

www.buzzfeednews.com[SEP]article[SEP]tasneemnashrulla[SEP]eat-babies-aoc-town-hall-pro-trump-troll-larouche	Elapsed time in seconds: 0.05031239986419678


In [32]:
curr_url_ix = 1
curr_url = df.url.values[curr_url_ix]
quotes = get_quotes(get_fulltext(curr_url)[0])

In [20]:
fname = get_fname(curr_url)
print(fname)
with open('./extracted_quotes/{}.jsonlist'.format(fname),'w+') as f:
    for res in quotes:
        json.dump(res, f)
        f.write('\n')

www.buzzfeednews.com[SEP]article[SEP]tasneemnashrulla[SEP]eat-babies-aoc-town-hall-pro-trump-troll-larouche


In [21]:
with open('./extracted_quotes/{}.jsonlist'.format(fname),'r') as f:
    data = f.readlines()
    print(len(data))
    
read_quotes = [json.loads(q.strip()) for q in data]
print(read_quotes[0])

28
{'quote': ['climate', 'change', 'is', 'a', 'hoax'], 'verb tokens': ['believes'], 'main verb': 'believes', 'subject tokens': ['the', 'pro', '-', 'Trump', 'LaRouche', 'PAC', ',', 'which', 'believes', ','], 'main subject': 'PAC', 'neg tokens': None, 'main neg': None, 'is neg': None}


In [22]:
read_quotes[1]

{'quote': ['climate', 'change', 'is', 'a', 'hoax', 'planted', 'the', 'troll'],
 'verb tokens': ['believes'],
 'main verb': 'believes',
 'subject tokens': ['the',
  'pro',
  '-',
  'Trump',
  'LaRouche',
  'PAC',
  ',',
  'which',
  'believes'],
 'main subject': 'PAC',
 'neg tokens': None,
 'main neg': None,
 'is neg': None}